In [1]:
#IMPORTING THE LIBRARIES
import random as rd
import numpy as np 
import matplotlib.pyplot as plt 
import statistics as stat
import math
import pandas as pd
from scipy import integrate as intg
from scipy.stats import linregress
import warnings
import scipy.stats as si
np.seterr(divide='ignore', invalid='ignore')
import scipy.integrate as integrate
import cmath
import scipy
warnings.filterwarnings('ignore')

import scipy.stats as st
import scipy.fft as fft
import scipy.interpolate as interpolate
import statsmodels.api as sm
from tqdm import tqdm
import time # to obtain the running time of some functions
from toolkit import generic_CF, genericFFT, odeRK4

## model

### MRDH

In [9]:
## solve for ChF of our proposed dynamics
def coef_D(tau, u, *var):
    '''
    Given u, compute D(tau,u) as a function of tau
    For our model, the coef for X0
    '''
    kappa = var[0]
    return 1j * u * np.exp(-kappa*tau)


def coef_C(u, *var):
    '''
    Given u, compute C(tau,u) as a function of tau
    For our model, the coef for m0
    Output:
        C_tau_u_func: function of time t
    '''
    T, kappa, kappam, omegam = var
    C_0_u = 0
    nU = len(u)
    
    def RHS_C(t,y):
        tmp1 = kappa*coef_D(t,u, kappa)
        tmp2 = -kappam*y
        tmp3 = 0.5*omegam**2*y**2
        return tmp1+tmp2+tmp3

    time, C_tau_u = odeRK4(RHS_C, 0, T, C_0_u, 30, nU)
    C_tau_u_func = interpolate.interp1d(time, C_tau_u, kind='cubic')
    
    return C_tau_u_func


## B(tau, u) for v_it
def coef_B(u, *var):
    '''
    Given u, compute B(tau,u) as a function of tau
    For our model, the coef for v0
    Output:
        B_tau_u_func: function of time t
    '''
    T, kappa, kappai, omegai, rhoi = var
    B_0_u = 0
    nU = len(u)
    
    def RHS_B(t,y):
        tmp1 = -kappai*y
        tmp2 = 0.5*coef_D(t,u, kappa)**2
        tmp3 = 0.5*omegai**2*y**2
        tmp4 = rhoi*omegai*coef_D(t,u, kappa)*y
        return tmp1+tmp2+tmp3+tmp4

    time, B_tau_u = odeRK4(RHS_B, 0, T, B_0_u, 30, nU)
    B_tau_u_func = interpolate.interp1d(time, B_tau_u, kind='cubic')
    
    return B_tau_u_func


def coef_A(u, *var):
    '''
    Given u, compute A(tau,u) as a function of tau
    For our model, the const term
    '''
    T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, kappa2, theta2, omega2, rho2, lamb, muJS, muJV = var
    A_0_u = 0
    nU = len(u)
    
    coef_B1_func_ = coef_B(u, T, kappa, kappa1, omega1, rho1)
    coef_B2_func_ = coef_B(u, T, kappa, kappa2, omega2, rho2)
    coef_C_func_  = coef_C(u, T, kappa, kappam, omegam)
    
    def RHS_A(t,y):
        tmp1 = kappa1*theta1*coef_B1_func_(t) ## B1
        tmp2 = kappa2*theta2*coef_B2_func_(t) ## B2
        tmp3 = kappam*thetam*coef_C_func_(t)
        ## for jump
        jump1 = 1/(1 - coef_D(t,u, kappa)*muJS)
        jump2 = 1/(1 - coef_B1_func_(t)*muJV)
        tmp4 = lamb * ( (jump1*jump2-1)- coef_D(t,u, kappa)*muJS) 
        return tmp1+tmp2+tmp3+tmp4

    time, A_tau_u = odeRK4(RHS_A, 0, T, A_0_u, 30, nU)
    A_tau_u_func = interpolate.interp1d(time, A_tau_u, kind='cubic')
    
    return A_tau_u_func


def ChF_Our(u,*var):
    '''
    ChF of log price XT under our model
    Parameters:
    params: model specific parameters, see Ye's research note
    ini_set: initial value for VIX, v1, v2, m
    '''
    params = var[0]
    T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, kappa2, theta2, omega2, rho2, lamb, muJS, muJV = params
    ini_set = var[1]
    VIX0, v10, v20, m0 = ini_set
    vix0 = np.log(VIX0) ## as the model is for vix = log(VIX)
    
    coef_A_func_  = coef_A(u, T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, kappa2, theta2, omega2, rho2, lamb, muJS, muJV)
    coef_B1_func_ = coef_B(u, T, kappa, kappa1, omega1, rho1)
    coef_B2_func_ = coef_B(u, T, kappa, kappa2, omega2, rho2)
    coef_C_func_  = coef_C(u, T, kappa, kappam, omegam)
    
    res = np.exp(coef_A_func_(T)+\
                 coef_B1_func_(T)*v10+\
                 coef_B2_func_(T)*v20+\
                 coef_C_func_(T)*m0+\
                 coef_D(T,u, kappa)*vix0)
    return res

### Shot noise

In [10]:
## solve for ChF of our proposed dynamics
def coef_A(tau, u, *var):
    '''
    Given u, compute A(tau,u) as a function of tau
    For our model, the coef for X0 (vix0S)
    '''
    kappa = var[0]
    return 1j * u * np.exp(-kappa*tau)


def coef_E(u, *var):
    '''
    Given u, compute E(tau,u) as a function of tau
    For our model, the coef for m0
    Output:
        E_tau_u_func: function of time t
    '''
    T, kappa, kappam, omegam = var
    E_0_u = 0
    nU = len(u)
    
    def RHS_E(t,y):
        tmp1 = kappa*coef_A(t,u, kappa)
        tmp2 = -kappam*y
        tmp3 = 0.5*omegam**2*y**2
        return tmp1+tmp2+tmp3

    time, E_tau_u = odeRK4(RHS_E, 0, T, E_0_u, 30, nU)
    E_tau_u_func = interpolate.interp1d(time, E_tau_u, kind='cubic')
    
    return E_tau_u_func


def coef_B(u, *var):
    '''
    Given u, compute B(tau,u) as a function of tau
    For our model, the coef for v0
    Output:
        B_tau_u_func: function of time t
    '''
    T, kappa, kappai, omegai, rhoi = var
    B_0_u = 0
    nU = len(u)
    
    def RHS_B(t,y):
        tmp1 = -kappai*y
        tmp2 = 0.5*coef_A(t,u, kappa)**2
        tmp3 = 0.5*omegai**2*y**2
        tmp4 = rhoi*omegai*coef_A(t,u, kappa)*y
        return tmp1+tmp2+tmp3+tmp4

    time, B_tau_u = odeRK4(RHS_B, 0, T, B_0_u, 30, nU)
    B_tau_u_func = interpolate.interp1d(time, B_tau_u, kind='cubic')
    
    return B_tau_u_func


def coef_C(u, *var):
    '''
    Given u, compute C(tau,u) as a function of tau
    For our model, the coef for L^s_0
    Output:
        C_tau_u_func: function of time t
    '''
    T, kappa, bs = var
    C_0_u = 0
    nU = len(u)
    
    def RHS_C(t,y):
        tmp1 = -bs*coef_A(t,u, kappa)
        tmp2 = -bs*y
        return tmp1+tmp2

    time, C_tau_u = odeRK4(RHS_C, 0, T, C_0_u, 30, nU)
    C_tau_u_func = interpolate.interp1d(time, C_tau_u, kind='cubic')
    
    return C_tau_u_func


def coef_D(u, *var):
    '''
    Given u, compute D(tau,u) as a function of tau
    For our model, the coef for L^v_0
    Output:
        D_tau_u_func: function of time t
    '''
    T, kappa, kappa1, omega1, rho1, bv = var
    D_0_u = 0
    nU = len(u)
    
    coef_B1_func_ = coef_B(u, T, kappa, kappa1, omega1, rho1)
    
    def RHS_D(t,y):
        tmp1 = -bv*coef_B1_func_(t)
        tmp2 = -bv*y
        return tmp1+tmp2

    time, D_tau_u = odeRK4(RHS_D, 0, T, D_0_u, 30, nU)
    D_tau_u_func = interpolate.interp1d(time, D_tau_u, kind='cubic')
    
    return D_tau_u_func


def coef_F(u, *var):
    '''
    Given u, compute F(tau,u) as a function of tau
    For our model, the const term
    '''
    T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV = var
    F_0_u = 0
    nU = len(u)
    
    coef_B_func_ = coef_B(u, T, kappa, kappa1, omega1, rho1)
    coef_C_func_ = coef_C(u, T, kappa, bs)
    coef_D_func_ = coef_D(u, T, kappa, kappa1, omega1, rho1, bv)
    coef_E_func_ = coef_E(u, T, kappa, kappam, omegam)
    
    def RHS_F(t,y):
        tmp1 = kappa1*theta1*coef_B_func_(t) ## B for v
        tmp2 = kappam*thetam*coef_E_func_(t)  ## E for m
        ## for jump
        jump_s = 1/(1 - (coef_A(t,u, kappa)+coef_C_func_(t)) * muJS)
        jump_v = 1/(1 - (coef_B_func_(t)+coef_D_func_(t)) * muJV)
        tmp3 = lamb * (jump_s*jump_v - 1) 
        return tmp1+tmp2+tmp3

    time, F_tau_u = odeRK4(RHS_F, 0, T, F_0_u, 30, nU)
    F_tau_u_func = interpolate.interp1d(time, F_tau_u, kind='cubic')
    
    return F_tau_u_func


def ChF_Our(u,*var):
    '''
    ChF of log price XT under our model
    Parameters:
    params: model specific parameters, see Ye's research note
    ini_set: initial value for VIX, v1, L^s,L^v, m
    '''
    params = var[0]
    T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV = params
    ini_set = var[1]
    VIX0, v10, Ls0,Lv0, m0 = ini_set
    vix0 = np.log(VIX0) ## as the model is for vix = log(VIX)
    
    coef_B_func_ = coef_B(u, T, kappa, kappa1, omega1, rho1)
    coef_C_func_ = coef_C(u, T, kappa, bs)
    coef_D_func_ = coef_D(u, T, kappa, kappa1, omega1, rho1, bv)
    coef_E_func_ = coef_E(u, T, kappa, kappam, omegam)
    coef_F_func_  = coef_F(u, T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV)
    
    res = np.exp(coef_A(T,u, kappa)*vix0+\
                 coef_B_func_(T)*v10+\
                 coef_C_func_(T)*Ls0+\
                 coef_D_func_(T)*Lv0+\
                 coef_E_func_(T)*m0+\
                 coef_F_func_(T))
    return res

## <a id='toc1_1_'></a>[Pricing via FFT](#toc0_)

In [11]:
def genericFFT_updated(r, T, params, ini_set, K, alpha, eta, n):
    '''
    Pricing via FFT using Carr and Madan's method with the specific ChF
    updated version from genericFFT() by Ali Hirsa

    params: T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, kappa2, theta2, omega2, rho2
    ini_set: VIX0, v10, v20, m0 
    '''
    
    N = 2**n
    
    # step-size in log strike space
    lda = (2 * np.pi / N) / eta
    
    # choice of beta
    #beta = np.log(S0)-N*lda/2 # the log strike we want is in the middle of the array
    beta = np.log(K) # the log strike we want is the first element of the array
    
    # forming vector x and strikes km for m=1,...,N
    km = np.zeros(N)
    xX = np.zeros(N)
    
    # discount factor
    df = np.exp(-r*T)
    
    nuJ = np.arange(N) * eta
    ## only modify here, by using your own ChF
    psi_nuJ = ChF_Our(nuJ - (alpha + 1) * 1j, params, ini_set) / ((alpha + 1j*nuJ)*(alpha+1+1j*nuJ))
    
    km = beta + lda * np.arange(N)
    w = eta * np.ones(N)
    w[0] = eta / 2
    xX = np.exp(-1j * beta * nuJ) * df * psi_nuJ * w
     
    yY = np.fft.fft(xX)
    cT_km = np.zeros(N) 
    multiplier = np.exp(-alpha * km) / np.pi
    cT_km = multiplier * np.real(yY)
    
    return km, cT_km

In [12]:
%%time
r,K, T, kappa = 0.03,40, 90/360, 5
kappam, thetam, omegam = 2.0, 3.0, 0.25
kappa1, theta1, omega1, rho1 = 2.0, 2.0, 3.0, 0.8 
bs,bv = 0.2,0.05
lamb, muJS, muJV = 1.5, 0.2, 0.2
params = [T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV]

VIX_0, v10, Ls0,Lv0, m0 = 40, 0.64, 0.,0., 3
ini_set = [VIX_0, v10, Ls0,Lv0, m0]

eta = 0.1
alpha = 1.0
n = 8

km, cT_km = genericFFT_updated(r, T, params, ini_set, K, alpha, eta, n)
np.round(cT_km[0],4)

CPU times: user 98.4 ms, sys: 0 ns, total: 98.4 ms
Wall time: 97.3 ms


1.5818

### <a id='toc1_1_1_'></a>[encapsule](#toc0_)

In [48]:
def Call_Pricer_FFT(r, T, K, params, ini_set):
    '''
    params: T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv, lamb, muJS, muJV
    ini_set: initial value of state variables
    '''
    eta, alpha, n = 0.25, 1.5, 8
    km, cT_km = genericFFT_updated(r, T, params, ini_set, K, alpha, eta, n)
    return np.round(cT_km[0], 6)

In [49]:
Call_Pricer_FFT(r, T, 35, params, ini_set)

2.18922

In [50]:
Call_Pricer_FFT(r, T, 45, params, ini_set)

1.182054

## <a id='toc1_2_'></a>[Pricing via MC](#toc0_)

In [51]:
def GeneratePathsEuler(NoOfPaths,NoOfSteps, params, ini_set, jump_param):
    '''
    Monte Carlo Simulation for our model as sanity check for FFT pricing method
    simulate vix_t = log VIX_t
    kappa: speed of mean-reverting
    gamma: vol of vol
    rho: correlation between 2 BMs
    theta: long-term variance
    '''
    T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv = params
    S_0, v10, Ls0,Lv0, m0 = ini_set
    muJ,eta,muJv = jump_param
    
    time = np.zeros([NoOfSteps+1])
    dt = T / float(NoOfSteps)

    ## diffusion terms
    B  = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps])  # BM for central tendency m
    Z1 = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps]) # BM for vix (log VIX)
    Z3 = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps]) # uncorrelated BM for v1
    Z_3 = np.zeros_like(Z3) ## correlated BM for v1
    
    ## jump terms
    ZPois = np.random.poisson(eta*dt,[NoOfPaths,NoOfSteps])    
    J = np.random.exponential(muJ,  [NoOfPaths,NoOfSteps])
    Jv= np.random.exponential(muJv,  [NoOfPaths,NoOfSteps])
    
    ## initiate paths for 4 processes
    V1 = np.zeros([NoOfPaths, NoOfSteps+1])
    m = np.zeros([NoOfPaths, NoOfSteps+1])
    X = np.zeros([NoOfPaths, NoOfSteps+1])
    Ls = np.zeros([NoOfPaths, NoOfSteps+1]) ## correlated BM for L^s
    Lv = np.zeros([NoOfPaths, NoOfSteps+1]) ## correlated BM for L^v
    
    V1[:,0] = v10
    m[:,0] = m0
    X[:,0] = np.log(S_0)
    Ls[:,0] = Ls0
    Lv[:,0] = Lv0
    
    for i in range(0, NoOfSteps):
        Z_3[:,i] = rho1 * Z1[:,i] + np.sqrt(1.0-rho1**2)*Z3[:,i]

        # Truncated boundary condition
        m[:, i+1] = m[:, i] + kappam*(thetam - m[:, i]) * dt + omegam* np.sqrt(m[:, i]) * dt**0.5*B[:, i]
        m[:, i+1] = np.maximum(m[:, i+1], 1e-4)
        V1[:,i+1] = V1[:,i] + kappa1*(theta1 - V1[:,i]) * dt + omega1* np.sqrt(V1[:,i]) * dt**0.5*Z_3[:,i] + Lv[:,i]
        V1[:,i+1] = np.maximum(V1[:,i+1], 1e-4)
        
        # shot noise
        Ls[:,i+1] = Ls[:,i] - bs*Ls[:,i] * dt + J[:, i]*ZPois[:,i]
        Lv[:,i+1] = Lv[:,i] - bv*Lv[:,i] * dt + Jv[:,i]*ZPois[:,i]

        ## inclusion of co-Jumps
        X[:,i+1] = X[:, i] + kappa *(m[:, i] - X[:, i]) * dt + np.sqrt(V1[:,i]) * dt**0.5*Z1[:,i] + Ls[:,i]
        time[i+1] = time[i] + dt

    # Compute exponent
    S = np.exp(X)
    return time, S

In [111]:
def GeneratePathsEuler(NoOfPaths,NoOfSteps, params, ini_set, jump_param):
    '''
    Monte Carlo Simulation for our model as sanity check for FFT pricing method
    simulate vix_t = log VIX_t
    kappa: speed of mean-reverting
    gamma: vol of vol
    rho: correlation between 2 BMs
    theta: long-term variance
    '''
    T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv = params
    S_0, v10, Ls0,Lv0, m0 = ini_set
    muJ, eta, muJv = jump_param
    
    time = np.zeros([NoOfSteps+1])
    dt = T / float(NoOfSteps)

    ## diffusion terms
    B  = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps])  # BM for central tendency m
    Z1 = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps]) # BM for vix (log VIX)
    Z3 = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps]) # uncorrelated BM for v1
    Z_3 = np.zeros_like(Z3) ## correlated BM for v1
    
    ## jump terms
    ZPois = np.random.poisson(eta*dt,[NoOfPaths,NoOfSteps])    
    J = np.random.exponential(muJ,  [NoOfPaths,NoOfSteps])
    Jv= np.random.exponential(muJv,  [NoOfPaths,NoOfSteps])
    
    ## initiate paths for 4 processes
    V1 = np.zeros([NoOfPaths, NoOfSteps+1])
    m = np.zeros([NoOfPaths, NoOfSteps+1])
    X = np.zeros([NoOfPaths, NoOfSteps+1])
    Ls = np.zeros([NoOfPaths, NoOfSteps+1]) ## correlated BM for L^s
    Lv = np.zeros([NoOfPaths, NoOfSteps+1]) ## correlated BM for L^v
    
    V1[:,0] = v10
    m[:,0] = m0
    X[:,0] = np.log(S_0)
    Ls[:,0] = Ls0
    Lv[:,0] = Lv0
    
    for i in range(0, NoOfSteps):
        Z_3[:,i] = rho1 * Z1[:,i] + np.sqrt(1.0-rho1**2)*Z3[:,i]
        
        # shot for variance
        Lv[:,i+1] = Lv[:,i] - bv*Lv[:,i] * dt + Jv[:,i]*ZPois[:,i]
        
        # Truncated boundary condition
        m[:, i+1] = m[:, i] + kappam*(thetam - m[:, i]) * dt + omegam* np.sqrt(m[:, i]) * dt**0.5*B[:, i]
        m[:, i+1] = np.maximum(m[:, i+1], 1e-4)
        V1[:,i+1] = V1[:,i] + kappa1*(theta1 - V1[:,i]) * dt + omega1* np.sqrt(V1[:,i]) * dt**0.5*Z_3[:,i] + (Lv[:,i+1] - Lv[:,i])
        V1[:,i+1] = np.maximum(V1[:,i+1], 1e-4)
        
        # shot noise for log VIX
        Ls[:,i+1] = Ls[:,i] - bs*Ls[:,i] * dt + J[:, i]*ZPois[:,i]
        
        ## inclusion of co-Jumps
        X[:,i+1] = X[:, i] + kappa *(m[:, i] - X[:, i]) * dt + np.sqrt(V1[:,i]) * dt**0.5*Z1[:,i] + (Ls[:,i+1] - Ls[:,i])
        time[i+1] = time[i] + dt

    # Compute exponent
    S = np.exp(X)
    return time, S, Ls, Lv

In [117]:
%%time
## set random seed for reproducibility
np.random.seed(10)

## set parameters for numerical illustration
r,K, T, kappa = 0.03,40, 90/360, 5
kappam, thetam, omegam = 2.0, 3.0, 0.25
kappa1, theta1, omega1, rho1 = 2.0, 2.0, 3.0, 0.8 
bs,bv = 0.2, 0.05
params = [T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv]

VIX_0, v10, Ls0,Lv0, m0 = 40, 0.64, 0.,0., 3
ini_set = [VIX_0, v10, Ls0,Lv0, m0]

muJS,eta,muJv = 0.2, 1.5, 0.2
jump_param = [muJS,eta,muJv]

NoOfSteps, NoOfPaths = 256, 500000 ## number of time steps and paths

## main test
t, VIX, Ls,Lv = GeneratePathsEuler(NoOfPaths,NoOfSteps, params, ini_set, jump_param)
call_mc = np.exp(-r*T)*np.mean(np.maximum(0, VIX[:,-1]-K))
print("Option via Monte Carlo: for strike %s the option premium is %6.4f" % (K, call_mc))

Option via Monte Carlo: for strike 40 the option premium is 1.5789
CPU times: user 1min 11s, sys: 2.47 s, total: 1min 13s
Wall time: 1min 13s


### <a id='toc1_2_1_'></a>[encapsule](#toc0_)

In [121]:
def Call_Pricer_MC(r, T, K, params, ini_set, jump_param):
    NoOfSteps, NoOfPaths = 256, 600000 ## number of time steps and paths
    t, VIX,Ls,Lv = GeneratePathsEuler(NoOfPaths,NoOfSteps, params, ini_set, jump_param)
    call_mc = np.exp(-r*T)*np.mean(np.maximum(0, VIX[:,-1]-K))
    return call_mc

In [122]:
## set parameters for numerical illustration
r, T, kappa = 0.03, 90/360, 5
kappam, thetam, omegam = 2.0, 3.0, 0.25
kappa1, theta1, omega1, rho1 = 2.0, 2.0, 3.0, 0.8 
bs,bv = 0.2,0.05
params = [T, kappa, kappam, thetam, omegam, kappa1, theta1, omega1, rho1, bs,bv]

## ini_set
VIX_0, v10, Ls0,Lv0, m0 = 40, 0.64, 0.,0., 3
ini_set = [VIX_0, v10, Ls0,Lv0, m0]

## jump_param
muJS,eta,muJv = 0.2, 1.5, 0.2
jump_param = [muJS,eta,muJv]

### <a id='toc1_2_2_'></a>[testing case](#toc0_)

In [ ]:
K = 40
call_ = np.zeros(10)
for i in tqdm(range(len(call_))):
    call_[i] = Call_Pricer_MC(r, T, K, params, ini_set, jump_param)
np.mean(call_)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
K = 35
call_ = np.zeros(10)
for i in tqdm(range(len(call_))):
    call_[i] = Call_Pricer_MC(r, T, K, params, ini_set, jump_param)
np.mean(call_)

In [ ]:
K = 45
call_ = np.zeros(10)
for i in tqdm(range(len(call_))):
    call_[i] = Call_Pricer_MC(r, T, K, params, ini_set, jump_param)
np.mean(call_)

## accuracy and stability

参考HPC 对应部分